In [18]:
# Import necessary packages and functions
from matplotlib.pylab import figure, semilogx, loglog, xlabel, ylabel, legend, title, subplot, show, grid
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
import pandas as pd
import sklearn.linear_model as lm
from sklearn import model_selection
import torch
import sys

# Add the path to the toolbox_02450 module
sys.path.append('../Tools')
from toolbox_02450 import rlr_validate, correlated_ttest, train_neural_net, draw_neural_net

# Import custom functions from data_preprocessing module
from data_preprocessing import *
dfjoint, dfRec, dfClas = dataPreprocess()
colNamesMeans, colNamesStd, colNamesExt, colNamesOther = getSpecificColNames()


In [11]:

n_left = 33 + 28 + 30 + 29
n_right = 4 + 2 + 3 + 5 + 1
n_total = n_right + n_left

p1 = (33 + 28 + 30 + 29)/n_total
p2 = (4 + 2 + 3 + 5)/n_total
p3 = 0
p4 = 0.075

lp = 1 - max(p1,p2,p3,p4)
ll = 1 - 1/120
lr = 1 - (n_left + n_right - 1) / 120

print(lp - ((n_left / n_total) * ll) - ((n_right / n_total) * lr))

-0.7574074074074073


In [19]:
# Extract data
mean = dfjoint.loc[:, colNamesMeans]
sde = dfjoint.loc[:, colNamesStd]
worst = dfjoint.loc[:, colNamesExt]
colNames = colNamesMeans + colNamesStd + colNamesExt

# Discretize time variable
time_discretized = []
for i in range(len(dfRec.iloc[:,0])):
    if dfRec['time'].iloc[i] <= 12:
        time_discretized.append('<1 years')
    elif dfRec['time'].iloc[i] <= 36:
        time_discretized.append('1-3 years')
    elif dfRec['time'].iloc[i] <= 72:
        time_discretized.append('>3-6 years')
    else:
        time_discretized.append('6+ years')

# Add discretized time variable to dataframe
dfRec['time_discretized'] = time_discretized


In [20]:

# Prepare data for classification
attributeNames = [u'Offset'] + colNames
X = dfRec.loc[:, colNames]
classLabels = dfRec.loc[:,"time_discretized"]
classNames = sorted(set(dfRec.loc[:,"time_discretized"]))

# Create a dictionary mapping class names to integer labels
classDict = dict(zip(classNames, range(4)))
y = np.asarray([classDict[value] for value in classLabels])

# Define data dimensions
N = X.shape[0]
M = len(attributeNames)
# M = M + 1
C = len(classNames)

# Add bias to input data
X = np.concatenate((np.ones((X.shape[0],1)), X), 1)


In [16]:
# 2 level cross validation, linear regression model

# Define number of folds for cross-validation
K = 10

# Create cross-validation partition for evaluation
CV = model_selection.KFold(K, shuffle=True)
#CV = model_selection.KFold(K, shuffle=False)

# Define lambda values to be tested
lambdas = np.power(10.,range(-5,9))

# Initialize variables to store errors and weights
Error_train = np.empty((K,1))
Error_test = np.empty((K,1))
Error_train_rlr = np.empty((K,1))
Error_test_rlr = np.empty((K,1))
Error_train_nofeatures = np.empty((K,1))
Error_test_nofeatures = np.empty((K,1))
w_rlr = np.empty((M,K))
mu = np.empty((K, M-1))
sigma = np.empty((K, M-1))
w_noreg = np.empty((M,K))
errors_lin_reg = np.empty((K,1))

# Loop over each fold in the cross-validation
k = 0
print(' ')
print('------------------------------------')
print('Linear regression model:')
print(' ')
for train_index, test_index in CV.split(X, y):

    # Extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]

    # Define the number of internal folds for cross-validation
    internal_cross_validation = 10

    # Validate the performance of the linear regression model using ridge regression
    opt_val_err, opt_lambda, mean_w_vs_lambda, train_err_vs_lambda, test_err_vs_lambda = rlr_validate(X_train, y_train, lambdas, internal_cross_validation)

    # Standardize the data based on the training set and save the mean and standard deviation
    # since they're part of the model
    mu[k, :] = np.mean(X_train[:, 1:], 0)
    sigma[k, :] = np.std(X_train[:, 1:], 0)

    X_train[:, 1:] = (X_train[:, 1:] - mu[k, :]) / sigma[k, :]
    X_test[:, 1:] = (X_test[:, 1:] - mu[k, :]) / sigma[k, :]

    # Save the generalization error for later statistics
    errors_lin_reg[k] = opt_val_err

    k += 1

    # Print the results of the current fold
    print('Outer fold {0}'.format(k))
    print('has optimal lambda: {0}'.format(opt_lambda))
    print('and optimal generalization error: {0}'.format(opt_val_err))

 
------------------------------------
Linear regression model:
 
Outer fold 1
has optimal lambda: 1000.0
and optimal generalization error: 1.2699102901348287
Outer fold 2
has optimal lambda: 100000000.0
and optimal generalization error: 1.3625279831668704
Outer fold 3
has optimal lambda: 100000000.0
and optimal generalization error: 1.3172608806155341
Outer fold 4
has optimal lambda: 100000000.0
and optimal generalization error: 1.3124708660504683
Outer fold 5
has optimal lambda: 100000000.0
and optimal generalization error: 1.341719254142665
Outer fold 6
has optimal lambda: 100000000.0
and optimal generalization error: 1.309148923006171
Outer fold 7
has optimal lambda: 100000000.0
and optimal generalization error: 1.3703819241338469
Outer fold 8
has optimal lambda: 100000000.0
and optimal generalization error: 1.3569186376537685
Outer fold 9
has optimal lambda: 100000000.0
and optimal generalization error: 1.2754145285900074
Outer fold 10
has optimal lambda: 100000000.0
and optimal g

In [19]:
# 2 level Cross validation, baseline

# Define number of outer and inner folds for cross-validation
K = 10
K_i = 10

# Create cross-validation partition for evaluation
CV = model_selection.KFold(K, shuffle=True)
#CV = model_selection.KFold(K, shuffle=False)

# Initialize variables to store results
mu = np.empty((K, M-1))
sigma = np.empty((K, M-1))
error_i = np.empty((K_i, 1))
error_o = np.empty((K, 1))


print(' ')
print('------------------------------------')
print('Baseline model:')
print(' ')
# Loop over outer cross-validation folds
k=0
for train_index, test_index in CV.split(X,y):
    
    # extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]
    
    
    CV_i = model_selection.KFold(K_i, shuffle=True)
    
    k_i = 0
    for train_index, test_index in CV_i.split(X_train,y_train):
        
        # extract training and test set for current inner CV fold
        X_train_i = X_train[train_index]
        y_train_i = y_train[train_index]
        X_test_i = X_train[test_index]
        y_test_i = y_train[test_index]
        
        mean_i = np.mean(y_train_i)
        error_i[k_i]=(1/len(y_test_i))*np.sum((y_test_i-mean_i)**2)
        #rykke error her op og find 10 inder errors
        #Error_i[k_i] = (1/len(y_test_i)*
        
        k_i+=1
    
    error_o[k] = min(error_i)
        
    # Standardize outer fold based on training set, and save the mean and standard
    # deviations since they're part of the model (they would be needed for
    # making new predictions) - for brevity we won't always store these in the scripts
    mu[k, :] = np.mean(X_train[:, 1:], 0)
    sigma[k, :] = np.std(X_train[:, 1:], 0)
    
    X_train[:, 1:] = (X_train[:, 1:] - mu[k, :] ) / sigma[k, :] 
    X_test[:, 1:] = (X_test[:, 1:] - mu[k, :] ) / sigma[k, :] 
    
    print('Outer fold {} generalization error: {}'.format(k+1,error_o[k]))

    k+=1

 
------------------------------------
Baseline model:
 
Outer fold 1 generalization error: [0.91666667]
Outer fold 2 generalization error: [0.90385455]
Outer fold 3 generalization error: [1.02119091]
Outer fold 4 generalization error: [0.83755455]
Outer fold 5 generalization error: [1.03083333]
Outer fold 6 generalization error: [0.93568367]
Outer fold 7 generalization error: [0.74893333]
Outer fold 8 generalization error: [0.75371399]
Outer fold 9 generalization error: [0.97616989]
Outer fold 10 generalization error: [0.7768668]


In [20]:
# ANN

# Create crossvalidation partition for evaluation
K = 10
K_i = 10
CV = model_selection.KFold(K, shuffle=True)
#CV = model_selection.KFold(K, shuffle=False)

# Initialize variables
mu = np.empty((K, M-1))
sigma = np.empty((K, M-1))
error_i = np.empty((K_i,1))

minerror=np.empty((K,1))
min_ind=np.empty((K,1))


k_o=1
for train_index, test_index in CV.split(X,y):
    
    # extract training and test set for current CV fold
    X_train = X[train_index]
    y_train = y[train_index]
    X_test = X[test_index]
    y_test = y[test_index]


    # Parameters for neural network classifier
    # K-fold crossvalidation
    CV = model_selection.KFold(K_i, shuffle=True)
    
    #print('Training model of type:\n\n{}\n'.format(str(model())))
    errors = np.empty((K,1)) # make a list for storing generalizaition error in each loop
    for (k, (train_index, test_index)) in enumerate(CV.split(X_train,y_train)): 
        #print('\nCrossvalidation fold: {0}/{1}'.format(k+1,K_i))  
        n_hidden_units = k+1      # number of hidden units
        n_replicates = 1        # number of networks trained in each k-fold
        max_iter = 10000
        
        # Define the model
        model = lambda: torch.nn.Sequential(
                            torch.nn.Linear(M, n_hidden_units), #M features to n_hidden_units
                            torch.nn.Tanh(),   # 1st transfer function,
                            torch.nn.Linear(n_hidden_units, 1), # n_hidden_units to 1 output neuron
                            # no final tranfer function, i.e. "linear output"
                            )
        loss_fn = torch.nn.MSELoss() # notice how this is now a mean-squared-error loss
        
        # Extract training and test set for current CV fold, convert to tensors
        X_train_i = torch.Tensor(X_train[train_index,:])
        y_train_i = torch.Tensor(y_train[train_index])
        X_test_i = torch.Tensor(X_train[test_index,:])
        y_test_i = torch.Tensor(y_train[test_index])
        
        # Train the net on training data
        net, final_loss, learning_curve = train_neural_net(model,
                                                           loss_fn,
                                                           X=X_train_i,
                                                           y=y_train_i,
                                                           n_replicates=n_replicates,
                                                           max_iter=max_iter)
        
        print('\n\tBest loss: {}\n'.format(final_loss))
        
        # Determine estimated class labels for test set
        y_test_est = net(X_test_i)
        
        errors[k]=final_loss # store error rate for current CV fold 
    
    minerror[k_o-1]=min(errors)
    min_ind[k_o-1] = np.argmin(errors)
    k_o += 1


print(' ')
print('------------------------------------')
print('ANN model:')
print(' ')    
for ii in range(10):
    print('For outer fold {} best generalization error is {} for h = {} '.format(ii+1,minerror[ii],min_ind[ii]+1))



	Replicate: 1/1
		Iter	Loss			Rel. loss


c:\ProgramData\Anaconda3\envs\py10\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([99])) that is different to the input size (torch.Size([99, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


		1000	1.6127735	0.0006505521
		2000	1.3156444	1.1597824e-05
		Final loss:
		2335	1.3135431	9.98293e-07

	Best loss: 1.3135430812835693


	Replicate: 1/1
		Iter	Loss			Rel. loss


c:\ProgramData\Anaconda3\envs\py10\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


		1000	1.913898	0.0010769494
		2000	1.358823	1.6142027e-05
		Final loss:
		2366	1.3557918	9.671845e-07

	Best loss: 1.3557918071746826


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		617	1.3317014	9.846809e-07

	Best loss: 1.331701397895813


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		559	1.3275521	9.877585e-07

	Best loss: 1.327552080154419


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		661	1.3484602	8.840393e-07

	Best loss: 1.3484601974487305


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		90	1.417433	1.6820445e-07

	Best loss: 1.4174330234527588


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		98	1.3419073	8.8835634e-07

	Best loss: 1.341907262802124


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		367	1.2969183	9.1917263e-07

	Best loss: 1.296918272972107


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		192	1.3080162	9.113739e-07

	Best loss: 1.3080161809921265


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final 

c:\ProgramData\Anaconda3\envs\py10\lib\site-packages\torch\nn\modules\loss.py:536: UserWarning: Using a target size (torch.Size([101])) that is different to the input size (torch.Size([101, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


		1000	1.3814336	3.9695037e-06
		2000	1.3746121	6.0705124e-06
		3000	1.3658043	6.8079016e-06
		4000	1.3572656	5.1819716e-06
		5000	1.3515283	2.8224983e-06
		Final loss:
		5661	1.3497292	9.71529e-07

	Best loss: 1.3497291803359985


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		289	1.3595004	9.645462e-07

	Best loss: 1.3595004081726074


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		929	1.4126822	9.2823495e-07

	Best loss: 1.4126821756362915


	Replicate: 1/1
		Iter	Loss			Rel. loss
		1000	1.4262264	1.2788144e-05
		2000	1.4052812	1.5438729e-05
		3000	1.3851348	1.19626575e-05
		4000	1.3729529	5.643719e-06
		Final loss:
		4931	1.3691095	9.577765e-07

	Best loss: 1.3691095113754272


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		247	1.3896384	8.578432e-07

	Best loss: 1.3896384239196777


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		525	1.3546152	9.680247e-07

	Best loss: 1.3546152114868164


	Replicate: 1/1
		Iter	Loss			Rel. loss
		Final loss:
		3

In [13]:
import sys
print(sys.path)

['c:\\Users\\Jakob Højgaard\\OneDrive - Danmarks Tekniske Universitet\\DTU\\8. Semester\\02450 Introduction to Machine Learning and Data Mining\\project_1\\02450intro_machine_learning\\Project 2\\Scripts', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\python310.zip', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\DLLs', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\lib', 'c:\\ProgramData\\Anaconda3\\envs\\py10', '', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\lib\\site-packages', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\lib\\site-packages\\win32', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\lib\\site-packages\\win32\\lib', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\lib\\site-packages\\Pythonwin', 'c:\\ProgramData\\Anaconda3\\envs\\py10\\lib\\site-packages\\IPython\\extensions', 'C:\\Users\\Jakob Højgaard\\.ipython', '../Tools']


In [13]:
#%% Statistical test for method setup II

# MSE are saved in variable:
    # errors_lin_reg for linear regression
    # error_o for baseline
    # minerror for ANN

# Generalization error differences
r_lin_base = errors_lin_reg - error_o
r_lin_ANN = minerror - errors_lin_reg
r_ANN_base = minerror - error_o

# significance level
alpha = 0.05

rho = 1/K

print(' ')
# Test lin reg vs baseline
print('Correlated t-test for linear regression and baseline')
p_val, conf_int = correlated_ttest(r_lin_base, rho, alpha)
print('p-value {} and confidence interval {}'.format(p_val, conf_int))
print(' ')

# Test ANN vs baseline
print('Correlated t-test for ANN and baseline')
p_val, conf_int = correlated_ttest(r_ANN_base, rho, alpha)
print('p-value {} and confidence interval {}'.format(p_val, conf_int))
print(' ')

# Test lin reg vs ANN
print('Correlated t-test for linear regression and ANN')
p_val, conf_int = correlated_ttest(r_lin_ANN, rho, alpha)
print('p-value {} and confidence interval {}'.format(p_val, conf_int))

 
Correlated t-test for linear regression and baseline
p-value 0.012789825807308003 and confidence interval (-1.1167083413544194, -0.1739114710822698)
 
Correlated t-test for ANN and baseline
p-value 2.165393111676037e-05 and confidence interval (0.29755289921442724, 0.5312778098179999)
 
Correlated t-test for linear regression and ANN
p-value 0.0002782461866609748 and confidence interval (0.6424045789944546, 1.4770459424746618)
